In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import xml
from pathlib import Path
from dataclasses import dataclass
from PIL import Image
import matplotlib.pyplot as plt
from ConfigSpace.read_and_write import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

sys.path.insert(1, os.path.join(os.getcwd()  , ''))

from nni.compression.pytorch import ModelSpeedup
from nni.compression.pytorch.utils.counter import count_flops_params

from utils import create_model, get_dataloader, parse_bool, Args
from  pruning_experiments import run_pruning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_workers = 16
torch.set_num_threads(num_workers)


# Preparation Steps:
- Make sure **pytorch, nni and tensorboard and ConfigSpace** are installed
- Run "Compressing MobileNetPruning.ipynb" to download data and obtain the pretrained model(s)

In [ ]:

with open('config_space.json', 'r') as f:
        json_string = f.read()
        config = json.read(json_string)
cfg = parse_bool(config.sample_configuration().get_dictionary())
args = Args(**cfg)

# Speed Up Experiments
args.agp_n_iters = 1
args.agp_n_epochs_per_iter = 1
args.finetune_epochs = 1
args.speed_up = True
args

In [ ]:
test_loss, test_acc, val_loss, val_acc, flops, params, init_dict = run_pruning(args)

In [ ]:
test_loss, test_acc, val_loss, val_acc, flops, params, init_dict

In [ ]:

print(f"Accuracy  {val_acc / init_dict['init_acc']}")
print(f"FLOPS:    {flops / init_dict['flops0']}")
print(f"Sparsity: {params / init_dict['params0']}")